In [2]:
import numpy as np
from scipy.signal import convolve2d

In [3]:
#Función grad_conjugado versión 2
#Dada una tolerancia arbitraria, x_0, A\in R^{nxn} definida positiva y b\in R^n.
def grad_conjugado_v2(A, b, x0, tol, max_iter=1000):
    #Definimos r_0 <- Ax_0-b, p_0 <- -r_0 y k <- 0
    r0 = A @ x0 - b
    p0 = -r0
    k = 0
    #Mejora #1, definimos rTr desde antes para evitar calcular el producto de
    #matrices varias veces. También, np.dot es mucho más eficiente.
    rTr0 = np.dot(r0, r0)
    #Aquí podemos hacer que nuesta k sea menor a un 
    #número de iteraciones máximo, pero mejor comparamos
    #el valor del residuo con el valor de la tolerancia
    #para decidir si seguir o no,es decir, si el valor
    #del residuo es suficientemente pequeño y la solución
    #se acerca cada vez más a la solución exacta.
    #Gracias a esto, después no necesitaremos el condicional
    #r_1 distinto de 0, ya que es implícito en el while
    while np.linalg.norm(r0) > tol and k < max_iter:
        #Mejora #2, calculamos A@p0 desde antes para evitar calcularlo 
        #varias veces en un solo ciclo, esto hará que se resuelva mucho más rápido
        Ap0 = A @ p0
        #Definimos el tamaño de paso
        #\alpha <- \frac{r_k^T * r_k}{p_k^T * A * p_k} igual usando np.dot en vez
        #de @
        alpha = rTr0 / np.dot(p0, Ap0)
        #Actualizamos nuestro vector de solución moviéndolo 
        #en alpha pasos hacia la direción p_0
        #x_1 <- x_0 + \alpha * p_0
        x1 = x0 + alpha * p0
        #Calculamos r_1 <- r_0 + \alpha * A * p_0
        r1 = r0 + alpha * Ap0
        #Mejora #3, análogo a la mejora 1, guardamos el valor de rTr en una 
        #variable para no calcularalo varias veces
        rTr1 = np.dot(r1, r1)
        #Calculamos el factor para actualizar la dirección de búsqueda
        #Para asegurar que las direcciones de búsqueda
        #Sean A-conjugadas
        #\beta = frac{r_1^T*r_1}{r_0^T * r_0}
        beta = rTr1 / rTr0
        #calculamos la nueva dirección de búsqueda p_0
        #p_0 <- -r_1 + \beta * p_0
        p0 = -r1 + beta * p0
        #Mejora #4
        #Sumamos 1 a la cantidad de iteraciones y re definimos r0, 
        #y de una vez redefinimos rTr0 PARA NO TENER QUE VOLVER A CALCULARLO
        r0, rTr0 = r1, rTr1
        k += 1

    return x1, k


In [12]:
def make_gaussian_kernel(size, sigma):
    """Genera un kernel gaussiano 2D con un tamaño y sigma dados."""
    # Crea un vector con distribución gaussiana
    x = np.linspace(-size // 2, size // 2, size)
    gauss_1d = np.exp(-x**2 / (2 * sigma**2))
    # Normaliza el vector para que la suma sea 1
    gauss_1d /= gauss_1d.sum()
    # Crea un kernel 2D con el producto externo del vector gaussiano con sí mismo
    gauss_2d = np.outer(gauss_1d, gauss_1d)
    # Normaliza el kernel 2D
    gauss_2d /= gauss_2d.sum()
    return gauss_2d

# Definir el tamaño del kernel y sigma
tamaño_kernel = 5  # Por ejemplo, un kernel de 5x5
sigma = 1.0  # Sigma para la distribución gaussiana

# Generar el kernel gaussiano
kernel = make_gaussian_kernel(tamaño_kernel, sigma)

In [13]:
import numpy as np
import cv2
from scipy.signal import convolve2d
from scipy.ndimage import gaussian_filter

def grad_conjugado_v2_deconv(A_func, At_func, b, x0, tol, max_iter):
    # Inicialización de variables
    r0 = A_func(x0) - b
    p0 = -r0
    k = 0
    rTr0 = np.dot(r0.ravel(), r0.ravel())
    
    while np.linalg.norm(r0) > tol and k < max_iter:
        Ap0 = A_func(p0)
        alpha = rTr0 / np.dot(p0.ravel(), Ap0.ravel())
        x1 = x0 + alpha * p0
        r1 = r0 + alpha * Ap0
        rTr1 = np.dot(r1.ravel(), r1.ravel())
        beta = rTr1 / rTr0
        p0 = -r1 + beta * p0
        r0, rTr0 = r1, rTr1
        k += 1
    
    return x1, k

def A_func(imagen):
    # Asegúrate de que la imagen de entrada es válida
    if imagen is None:
        raise ValueError("La imagen de entrada es None")
    # Realiza la operación de convolución y asegúrate de que el resultado no es None
    resultado = convolve2d(imagen, kernel, mode='same', boundary='wrap')
    if resultado is None:
        raise ValueError("El resultado de la convolución es None")
    return resultado

def At_func(imagen):
    # Asegúrate de que la imagen de entrada es válida
    if imagen is None:
        raise ValueError("La imagen de entrada es None")
    # Realiza la operación de convolución transpuesta y asegúrate de que el resultado no es None
    resultado = convolve2d(imagen, kernel[::-1, ::-1], mode='same', boundary='wrap')
    if resultado is None:
        raise ValueError("El resultado de la convolución transpuesta es None")
    return resultado

# Carga de la imagen clara y la imagen borrosa
imagen_clara = cv2.imread('image1.jpg', 0)
imagen_borrosa = cv2.imread('img1_b.png', 0)

# Generación del kernel gaussiano
tamaño_kernel = 5
sigma = 1.0
# La función gaussian_filter automáticamente maneja la generación del kernel
imagen_desenfocada = gaussian_filter(imagen_clara, sigma=sigma)

# Restauración de la imagen desenfocada y la imagen borrosa
tol = 1e-6
max_iter = 10000
x0 = np.zeros_like(imagen_borrosa, dtype=np.float32)

# Aquí deberás definir cómo se genera tu kernel gaussiano para las funciones A_func y At_func
# Por ejemplo, utilizando scipy.ndimage o cualquier otro método que prefieras

imagen_restaurada_desde_clara, _ = grad_conjugado_v2_deconv(A_func, At_func, imagen_desenfocada, x0, tol, max_iter)
imagen_restaurada_desde_borrosa, _ = grad_conjugado_v2_deconv(A_func, At_func, imagen_borrosa, x0, tol, max_iter)

# Guardar o mostrar las imágenes resultantes
cv2.imwrite('imagen_desenfocada.jpg', imagen_desenfocada)
cv2.imwrite('imagen_restaurada_desde_clara.jpg', imagen_restaurada_desde_clara)
cv2.imwrite('imagen_restaurada_desde_borrosa.jpg', imagen_restaurada_desde_borrosa)


ValueError: convolve2d inputs must both be 2-D arrays